In [1]:
!pip install mediapipe opencv-python

In [56]:
import cv2
import mediapipe as mp
import numpy as np 
mp_draw = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose 

# Calculate angle

In [57]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

# Squats AI Gym Instructor

In [59]:
cap = cv2.VideoCapture(0)

# Squats counter variables
counter = 0 
timer = 30
stage = None
start = False #whether user has started exercising


## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB since mediapipe only accepts image in this format
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks from image
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get (x,y) coordinates
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            nose = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            
            # Calculate angle
            angle1 = calculate_angle(hip, knee, ankle)
            angle2 = calculate_angle(nose,knee,hip)
            angle3 = calculate_angle(nose,hip,knee)
            # 3 angles used to maximise accuracy of posture and strictness of our program
       
            # Display corrections on live feed to guide users to correct posture
            if angle1>120 and angle1<160 and stage == "up" and counter>0:
                cv2.rectangle(image, (5,130), (300,250), (255,255,255), -1)
                cv2.putText(image, 'Go Lower!', (12,200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 1, cv2.LINE_AA)
            if angle2>50 and angle3<120 and stage == "up" and counter>0:
                cv2.rectangle(image, (5,260), (300,380), (255,255,255), -1)
                cv2.putText(image, 'Back Bent!', (12,330), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 1, cv2.LINE_AA)
            if angle1 < 120 and angle1 > 70 and angle2 < 50 and angle3<120 and angle3>60 :
                if counter == 0:
                    start = True
                stage = "down"
            if angle1 > 150 and angle3 > 150 and stage =="down":
                stage="up"
                counter +=1
                       
        except:
            pass
        
        
        # when our program senses the user starting the exercise, it will display the following message for 30 ticks
        if timer>0 and start == True:
            cv2.putText(image, 'Beginning Exercise', (20,240), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2, cv2.LINE_AA)
            timer-=1
            
        #display counter on live feed
        cv2.putText(image, str(counter), 
                    (10,100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 3, (255,255,255), 2, cv2.LINE_AA)
        
        #display stage on live feed
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA) 
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # display joints and connections
        mp_draw.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_draw.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_draw.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('x'):
            break

    cap.release()
    cv2.destroyAllWindows()